In [34]:
import pandas as pd
import datetime
import random
from datetime import datetime, timedelta
from requests import get
import q

class Colection:
    def __init__(self):
        self.user = input('Inserta tu nombre ') #TODO pick from users
        self.words = pd.read_csv(directory+'\words.csv', sep = ';') #TODO words_user add
        self.suggestions = pd.read_csv(directory+'\words_suggestions.csv', sep = ';')[['spanish','english']]
        #self.words = pd.DataFrame()
        self.levels = {x : 2**x for x in range(9)}
        self.checks = {4 : {'er_mar' : 0.5, 'back' : 1}, 
                       5 : {'er_mar' : 0.3, 'back' : 1}, 
                       6 : {'er_mar' : 0.2, 'back' : 2}, 
                       7 : {'er_mar' : 0.1, 'back' : 2}, 
                       8 : {'er_mar' : 0.05, 'back' : 2}}
        
    def scrape_meaning(self, word):
        while word[-1] == ' ':
            word = word[:-1]
        
        if len(word.split(' ')) > 1: #get rid of the article
            word = word.split(' ')[-1]
        
        word = word.replace(' ','')
        
        url = f'https://www.wordreference.com/definicion/{word}'
        
        my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"}
        html = str(get(url, headers = my_headers).text)
        html_split = html.split("</h3><ol class='entry'><li>")[1]

        def_list = []
        for x in html_split.split('</span><br>')[0].split('<li>'):
            if ':<br>' in x and '<span' not in x.split(':<br>')[0] and 'loc.' not in x:
                def_list.append(x.split(':<br>')[0])
                
        for num, definition in enumerate(def_list):
            print(f'definition {num}: {definition}')
            
        user_inp = input('Si quieres usar una de las definiciones, presione el numero de la definición')
        
        try:
            user_inp = int(user_inp)
        except:
            pass
        
        if type(user_inp) is int and -1 < user_inp < len(def_list):
            return def_list[user_inp]
        
        return None
        
    def add_suggested_words(self, amount = 10):
        d = self.suggestions.set_index('spanish').to_dict()['english']
        words = list(d.keys())
        random.shuffle(words)
        
        checked_words = []
        
        for word in words:
            if amount != 0:
                if input(f'Si quieres aprender {word}, (en ingles: {d[word]})?, presione clave "1"') == '1':
                    
                    self.add_word(word=word)
                    checked_words.append(word)
                    amount -= 1  
                else:
                    checked_words.append(word)
            else:
                break
                    
        self.suggestions = self.suggestions[~self.suggestions.spanish.isin(checked_words)]
        self.suggestions.to_csv(directory+'\words_suggestions.csv' ,sep= ';', index=False)
            
    def add_word(self, word = None):      
        #Standard dict - this part will be the same for every record
        columns_none = ['last_review','error_margin','trials','level','errors'] #Q do we need a last review?
        record = {c : 0 for c in columns_none}
        record['next_review'] = datetime.now()
        
        #If a word was suggested with add_suggested_word function no user input is requiered
        if word:
            record['word'] = word
        else:
            record['word'] = input('Que palabra quieres agregar? ')
        
        #Try to get scrape definiton otherwise use user input
        scrape_description = self.scrape_meaning(record['word'])
        if scrape_description:
            record['description'] = scrape_description  
        else:
            record['description'] = input('Como describes la palabra? ')
        
        #Update the word list and save
        self.words = self.words.append(record, ignore_index=True)
        self.save_words()
           
    def save_words(self):
        self.words.to_csv(directory+'\words.csv' ,sep= ';', index=False)
        
    def generic_test(self, d):
        errors = list(d.keys())
        
        while len(errors) > 0: #Make seperate function of this part if the practice will be added
            new_errors = []
            random.shuffle(errors)
            for key in errors:
                print(d[key])
                if input('¿Qué significa eso? ') != key:
                    print (f'la respuesta correcta es: {key}')
                    new_errors.append(key)
                print('')
            errors = new_errors 
        
    def practice_words(self, error_margin = 0.6, amount =10 , min_trials = 10):
        df =self.words[(self.words.error_margin > error_margin) & (self.words.trials >= min_trials)]
        df = df.sample(n=amount)
        d = df.set_index('word').to_dict()['description']
        
        
        
        self.generic_test(d)
        
                   
    def test_words(self):
        def update_row(word, is_error):
            d = self.words[self.words.word == word].to_dict('records')[0]
            d['trials'] += 1
            if is_error is True:
                d['errors'] += 1

            d['error_margin'] = d['errors'] / d['trials']
            
            d['last_review'] = datetime.now()
            tdelta = self.levels[d['level']]
            d['next_review'] = datetime.now() + timedelta(days=tdelta)            
            
            if d['level'] in self.checks.keys() and d['error_margin'] >= self.checks[d['level']]['er_mar']:
                d['level'] = self.checks[d['level']]['back']
            elif d['level'] < 8:
                d['level'] +=1
                
            #Change the row
            self.words.loc[self.words['word'] == d['word']] = [pd.Series(d)]
        
        self.words.next_review = pd.to_datetime(self.words.next_review)
        test_words = self.words[self.words.next_review <= datetime.now()]

        d = test_words.set_index('word').to_dict()['description']
        
        errors = []
        preguntas = list(d.keys())
        random.shuffle(preguntas)
        
        for key in preguntas:
            print(d[key])
            if input('¿Qué significa eso? ') != key:
                print (f'la respuesta correcta es: {key}')
                print('')
                errors.append(key)
                update_row(key,True)
            else:
                update_row(key,False)     
        self.save_words()
        
        d = {k:v for k, v in d.items() if k in errors}
        self.generic_test(d)
        
            
directory = #insert the directory
colection = Colection()

Inserta tu nombre geert


In [12]:
new_word = []
for w in colection.suggestions.spanish:
    while w[0] == ' ':
        w = w[1:]
    
    while w[-1] == ' ':
        w = w[:-1]
        
    new_word.append(w)
    
colection.suggestions.spanish = new_word


colection.suggestions.to_csv(directory+'\words_suggestions.csv' ,sep= ';', index=False)

In [32]:
colection.practice_words(min_trials=1)

 Acción y resultado de reunir y guardar cosas en cantidad
¿Qué significa eso? el almacenamiento

Que tiene poca disposición para hacer algo que requiere esfuerzo o constituye una obligación, especialmente trabajar.
¿Qué significa eso? perezoso

 Ángulo entrante que se forma donde se encuentran dos o tres superficies, especialmente dos paredes.
¿Qué significa eso? el rincón

tr. Oler con atención, aplicando el olfato repetidas veces
¿Qué significa eso? olfatear

f. En un centro docente,  sala destinada a la enseñanza
¿Qué significa eso? el aula

Pieza de tela más o menos gruesa que cubre las sábanas y mantas de una cama y sirve de adorno y de abrigo.
¿Qué significa eso? la colcha

tr. y prnl. Dar calor,  elevar la temperatura. También intr.
¿Qué significa eso? calentar

 Quitar la paz o tranquilidad a alguien
¿Qué significa eso? turbar
la respuesta correcta es: perturbar

 Mala costumbre adquirida por alguna circunstancia y que es difícil de eliminar.
¿Qué significa eso? el resabio

m. 

In [8]:
colection.save_words()

In [84]:
colection.add_word()

Que palabra quieres agregar? la harina
Como describes la palabra?  Polvo que resulta de la molienda de los cereales y otras semillas.


In [86]:
colection.save_words()

In [259]:
colection.words

,description,error_margin,errors,last_review,level,next_review,trials,word


In [40]:
colection.test_words()

In [35]:
colection.add_suggested_words(amount = 1)

Si quieres aprender el queso, (en ingles:  cheese )?, presione clave "1"2
Si quieres aprender el mineral, (en ingles:  mineral )?, presione clave "1"2
Si quieres aprender el magistrado, (en ingles:  magistrate )?, presione clave "1"2
Si quieres aprender reclinar, (en ingles: tilt. recline)?, presione clave "1"1
definition 0: tr. y prnl. Inclinar sobre algo que sirve de apoyo
Si quieres usar una de las definiciones, presione el numero de la definición0


In [14]:
url = f'https://www.wordreference.com/definicion/rechazar'

my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"}
html = str(get(url, headers = my_headers).text)
html_split = html.split("</h3><ol class='entry'><li>")[1]

def_list = []
for x in html_split.split('</span><br>')[0].split('<li>'):
    if ':<br>' in x and '<span' not in x.split(':<br>')[0] and 'loc.' not in x:
        def_list.append(x.split(':<br>')[0])

for num, definition in enumerate(def_list):
    print(f'definition {num}: {definition}')

definition 0: tr. No aceptar
definition 1:  Resistir un cuerpo a otro obligándolo a retroceder en su movimiento
definition 2:  Resistir


In [4]:
import json

with open(r'C:\Users\geert\OneDrive\Documenten\palabras\palabras.json', "r") as read_file:
    d = json.load(read_file)
    
with open(r'C:\Users\geert\OneDrive\Documenten\palabras\palabras_historico.json', "r") as read_file:
    d_hist = json.load(read_file)




In [5]:
d.update(d_hist)


len(d)

334

In [6]:
teller = 0
tdelta = 0

for word in d.keys():
    if teller % 15 == 0:
        tdelta += 1
    
    columns_none = ['last_review','error_margin','trials','level','errors'] #Q do we need a last review?
    record = {c : 0 for c in columns_none}
    record['next_review'] = datetime.now() + timedelta(days=tdelta) - timedelta(days=1)
    record['word'] = word
    record['description'] = d[word]
    colection.words = colection.words.append(record, ignore_index=True)
    colection.save_words()
    
    teller += 1

    